In [2]:
import pandas as pd
from arcgis.features import GeoAccessor
from arcgis.geometry import Geometry
import arcpy
import os

In [3]:
human_movement_origins_fc = '..\\data\\interim\\interim.gdb\\trips_raw'
geographic_area_fc = '..\\data\\raw\\raw.gdb\\blocks'
gdb = '../data/interim/interim.gdb'

In [4]:
hmn_mvmt_intersect = arcpy.analysis.Intersect([human_movement_origins_fc, geographic_area_fc], os.path.join(gdb, 'hmn_mvmt_intersect'))[0]
hmn_mvmt_intersect

'..\\data\\interim\\interim.gdb\\hmn_mvmt_intersect'

In [26]:
df_int = GeoAccessor.from_featureclass(hmn_mvmt_intersect)
df_int.head()

,OBJECTID,FID_trips_raw,travel_distance_miles,travel_time_minutes,store_locnum,FID_block_groups,ID,NAME,SHAPE
0,1,59437,261.898556,270.266667,718176685,153,060930004001,060930004.001,"{""x"": -122.56721299999998, ""y"": 41.86095800000..."
1,2,583754,311.607615,272.600000,219649035,153,060930004001,060930004.001,"{""x"": -122.56785399999995, ""y"": 41.90621900000..."
2,3,626251,311.607615,272.600000,723838910,153,060930004001,060930004.001,"{""x"": -122.56785399999995, ""y"": 41.90621900000..."
3,4,568161,271.754121,269.116667,403048346,89,410333616007,410333616.007,"{""x"": -123.71685499999995, ""y"": 42.05008500000..."
4,5,680059,271.754121,269.116667,970020814,89,410333616007,410333616.007,"{""x"": -123.71685499999995, ""y"": 42.05008500000..."


In [30]:
df_int = df_int[['travel_distance_miles', 'travel_time_minutes', 'store_locnum', 'ID', 'SHAPE']].copy()

In [31]:
df_int.columns = ['trip_distance_miles', 'trip_time_minutes', 'destination_id', 'origin_id', 'SHAPE']
df_int.head()

,trip_distance_miles,trip_time_minutes,destination_id,origin_id,SHAPE
0,261.898556,270.266667,718176685,060930004001,"{""x"": -122.56721299999998, ""y"": 41.86095800000..."
1,311.607615,272.600000,219649035,060930004001,"{""x"": -122.56785399999995, ""y"": 41.90621900000..."
2,311.607615,272.600000,723838910,060930004001,"{""x"": -122.56785399999995, ""y"": 41.90621900000..."
3,271.754121,269.116667,403048346,410333616007,"{""x"": -123.71685499999995, ""y"": 42.05008500000..."
4,271.754121,269.116667,970020814,410333616007,"{""x"": -123.71685499999995, ""y"": 42.05008500000..."


In [32]:
df_int.spatial.to_featureclass('D:\\projects\\geoai-retail\\data\\interim\\interim.gdb\\trips')

'D:\\projects\\geoai-retail\\data\\interim\\interim.gdb\\trips'

In [44]:
df_count = df_int[['destination_id', 'origin_id', 'SHAPE']].groupby(['destination_id', 'origin_id']).count()
df_count.columns = ['count']
df_count.head()

count
destination_id origin_id          
105830012      410019502002      1
               410050201003      4
               410050202003      1
               410050202004      3
               410050203021      7

In [45]:
df_mean = df_int[['destination_id', 'origin_id', 'trip_distance_miles', 'trip_time_minutes']].groupby(['destination_id', 'origin_id']).mean()
df_mean.head()

trip_distance_miles  trip_time_minutes
destination_id origin_id                                           
105830012      410019502002           304.353730         276.283333
               410050201003            10.867934          26.316667
               410050202003            10.502413          28.416667
               410050202004            12.473609          35.594444
               410050203021            17.355868          30.323810

In [46]:
df_stats = df_count.join(df_mean)

In [47]:
df_stats.reset_index(inplace=True)
df_stats.head()

,destination_id,origin_id,count,trip_distance_miles,trip_time_minutes
0,105830012,410019502002,1,304.353730,276.283333
1,105830012,410050201003,4,10.867934,26.316667
2,105830012,410050202003,1,10.502413,28.416667
3,105830012,410050202004,3,12.473609,35.594444
4,105830012,410050203021,7,17.355868,30.323810


In [48]:
df_stats.spatial.to_table('D:\\projects\\geoai-retail\\data\\interim\\interim.gdb\\trips_stats')

'D:\\projects\\geoai-retail\\data\\interim\\interim.gdb\\trips_stats'

In [49]:
df_stats.trip_distance_miles.median()

9.651212043375